## Functions

In [ ]:
from financial_data_loading import *

### Function Area 1: Risk Level

In [ ]:
# Cluster stocks based on their returns --> Low risk: first 3rd with lowest volatility, medium: first two, high: all
def risk_sorting(business_df, risk_level):
    std_sorted_business = business_df.sort_values(by="volatility", ascending = True)
    if risk_level.lower() == "low":
        risk_selected_df = std_sorted_business[:int(len(business_df)/3)]
    elif risk_level.lower() == "medium":
        risk_selected_df = std_sorted_business[:int((len(business_df)/3)*2)]
    else:
        risk_selected_df = business_df
    
    return risk_selected_df

### Function area 2: Business criteria

In [ ]:
from business import *

### Function area 3: Sustainability criteria

In [ ]:
from sustain import *

## Function Area 4: Find Buy recommendations

In [ ]:
from recommendation import *

## Data Import
All function that import the data can be found in the file "financial_data_loading.py"

In [ ]:
business_df, sustain_df, recommendation_df = load_data_csv();

In [ ]:
sp_return = load_retrun_time_series(y=2015)

### The overall stock dataframe will be filtered to only contain stocks that are contained in the Return Dataframe

In [ ]:
business_df = business_df[business_df["symbol"].isin(list(sp_return))]

## Volatility, returns, Sharpe Ratio

### Calculating volatility = annual standard deviation
Annualised volatility is calculated as the annualised standard deviation of each stock's return

In [ ]:
return_data = sp_return[sp_return.index > "2015-01-01"]
annual_std = []
for stock in return_data.columns:
    annual_std.append(np.std(return_data[stock])* 252 ** 0.5)
    
business_df["volatility"] = annual_std

### Determining yearly returns with help of log-returns

In [ ]:
# Creating log-return columns
log_return_list = []
return_data = sp_return.copy()
col_limit = return_data.shape[1]
for stock in return_data.columns[:col_limit]:
    return_data[f"{stock}_log_return"] =  np.log(1 + return_data[stock])
    
#Creating list with all log-returns
for stock in return_data.columns[:col_limit]:
    log_return_list.append(np.mean(return_data[f"{stock}_log_return"]))
    
# Annualising log-returns
for i, item in enumerate(log_return_list):
    log_return_list[i] = log_return_list[i] * 250

    # Adding annualised returns to DF
business_df["yearly_return"] = log_return_list

### Calculating sharpe-ratio based (assumption: risk-free rate is currently 0)

In [ ]:
business_df["sharpe_ratio"] = business_df["yearly_return"] / business_df["volatility"]

## Filter companies with Buy Recommendation
In the following cell, all companies with a Buy recommendation will be filtered from the overall stock dataframe

In [ ]:
# Show comapanies which have more buy recommendations then sell or neutral
# import from the file recoommendation.py the function recom
# gives the dataframe with all the companies for which the descripted is TRUE
recom_dict = {}
for ticker in list(recommendation_df.columns):
    recomm = recom(ticker,recommendation_df,m = 6)
    recom_dict[ticker] = recomm

recom_df = pd.DataFrame(recom_dict, index=["Rank"])
psoitiv_recom = recom_df[recom_df > 0]
psoitiv_recom.dropna(axis=1)

# Only keep the companies with a postive recommendation score
business_df_recommend = business_df[business_df["symbol"].isin(list(psoitiv_recom.dropna(axis=1)))]
business_df_recommend = business_df_recommend.set_index("symbol", drop = False)

## Interface

#### Running the following section will trigger the interface of the investment advisor app

In [ ]:
import tkinter as tk

def create_result(business_df_recommend = business_df_recommend):

    growth = get_growth_comp(business_df_recommend, option_Question4.get())
    div = get_div_comp(business_df_recommend, option_Question5.get())
    business_df_recom = business_df_recommend.copy()
    business_df_recom = (business_df_recom[(business_df_recom["symbol"].isin(div)) & (business_df_recom["symbol"].isin(growth))])

    nuc = nuclear(sustain_df,option_Question6.get())
    env = environment(sustain_df,option_Question7.get())
    soc = social(sustain_df,option_Question8.get())

    new_sustain = (sustain_df[(sustain_df["symbol"].isin(nuc)) & (sustain_df["symbol"].isin(env))
                              & (sustain_df["symbol"].isin(soc))])

    sus_list = list(new_sustain["symbol"])

    FINAL = business_df_recom[business_df_recom["symbol"].isin(sus_list)]
    
    FINAL = risk_sorting(FINAL, risk_level = option_Risk.get())
    
    FINAL.sort_values(by = "sharpe_ratio", ascending = False, inplace = True)

    if len(FINAL) >= 3:
        output_1 = FINAL.longName.iloc[0] 
        output_2 = FINAL.longName.iloc[1]
        output_3 = FINAL.longName.iloc[2]
    
    elif len(FINAL) == 2:
        output_1 = FINAL.longName.iloc[0] 
        output_2 = FINAL.longName.iloc[1]
        output_3 = ""
        
    elif len(FINAL) == 1:
        output_1 = FINAL.longName.iloc[0] 
        output_2 = ""
        output_3 = ""

    else:
        output_1 = ""
        output_2 = ""
        output_3 = ""
        
        
    if len(output_1) > 0 and len(output_2) > 0 and len(output_3) > 0:
        lbl_result["text"] = f"First recommended stock: {output_1}\nSecond recommended stock: {output_2}\nThird recommended stock: {output_3}"
    
    elif len(output_1) > 0 and len(output_2) > 0:
        lbl_result["text"] = f"First recommended stock: {output_1}\nSecond recommended stock: {output_2}"
    
    elif len(output_1) > 0:
        lbl_result["text"] = f"Recommended stock: {output_1}"
    
    else:
        lbl_result["text"] = f"Sorry, we could not find a stock that matches your criteria.\nPlease change your selection"
window = tk.Tk()
window.title("Your Investment Selection Advisor")

frm_entry = tk.Frame(master=window)



#Question 1 = Risk Level
#1.1 Set first input field with dropdown and label
option_Risk = tk.StringVar(master = frm_entry)
options_risk = ("Low","Medium","High")
menu_risk = tk.OptionMenu(frm_entry,option_Risk,*options_risk)
menu_risk.grid(row=0, column=1, sticky="w")
option_Risk.set("")

#1.2 Set label name for first input field
lbl_name = tk.Label(master=frm_entry, text="How high can the risk of your investment be?")
lbl_name.grid(row=0, column=0, sticky="e")


#Question 4: Growth company - yes/ no
#4.1 Set 4th input field with dropdown and label
option_Question4 = tk.StringVar(master = frm_entry)
options_question4 = ("Yes","No")
menu_question4 = tk.OptionMenu(frm_entry,option_Question4,*options_question4)
menu_question4.grid(row=3, column=1, sticky="w")
option_Question4.set("")

#4.2 Set label name for second input field
lbl_name = tk.Label(master=frm_entry, text="Do you want to invest in a company with an perspective of growth compared to a compay wich is established in the market?")
lbl_name.grid(row=3, column=0, sticky="e")


#Question 5: Dividend payments
#5.1 Set 5th input field with dropdown and label
option_Question5 = tk.StringVar(master = frm_entry)
options_question5 = ("Yes","No")
menu_question5 = tk.OptionMenu(frm_entry,option_Question5,*options_question5)
menu_question5.grid(row=4, column=1, sticky="w")
option_Question5.set("")

#5.2 Set label name for second input field
lbl_name = tk.Label(master=frm_entry, text="Is it important to you that your investment pays regulary small amounts of money, even though it may grow slower?")
lbl_name.grid(row=4, column=0, sticky="e")

#Question 6: Nuclear power
#6.1 Set 5th input field with dropdown and label
option_Question6 = tk.StringVar(master = frm_entry)
options_question6 = ("Yes","No")
menu_question6 = tk.OptionMenu(frm_entry,option_Question6,*options_question6)
menu_question6.grid(row=5, column=1, sticky="w")
option_Question6.set("")

#6.2 Set label name for second input field
lbl_name = tk.Label(master=frm_entry, text="Are you willing to invest in a company that uses nuclear power? ")
lbl_name.grid(row=5, column=0, sticky="e")


#Question 7: Environmental importance
#7.1 Set 7th input field with dropdown and label
option_Question7 = tk.StringVar(master = frm_entry)
options_question7 = ("Yes","No")
menu_question7 = tk.OptionMenu(frm_entry,option_Question7,*options_question6)
menu_question7.grid(row=6, column=1, sticky="w")
option_Question7.set("")

#7.2 Set label name for 7th input field
lbl_name = tk.Label(master=frm_entry, text="Are you concerned about environmental impacts of the companies? ")
lbl_name.grid(row=6, column=0, sticky="e")


#Question 8: Social importance
#8.1 Set 8th input field with dropdown and label
option_Question8 = tk.StringVar(master = frm_entry)
options_question8 = ("Yes","No")
menu_question8 = tk.OptionMenu(frm_entry,option_Question8,*options_question6)
menu_question8.grid(row=7, column=1, sticky="w")
option_Question8.set("")

#7.2 Set label name for 7th input field
lbl_name = tk.Label(master=frm_entry, text="Are you concerned about social impacts of the companies? ")
lbl_name.grid(row=7, column=0, sticky="e")

btn_convert = tk.Button(
    master=window,
    text="Find your best stocks",
    command = create_result
)

lbl_result = tk.Label(master=window, text="Result")

# Set-up the layout using the .grid() geometry manager
frm_entry.grid(row=0, column=0, padx=10)
btn_convert.grid(row=2, column=1, pady=10)
lbl_result.grid(row=3, column=1, padx=10)

# Run the application
window.mainloop()